# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7e2c8a1e80>
├── 'a' --> tensor([[-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710]])
└── 'x' --> <FastTreeValue 0x7f7e2c8a1f70>
    └── 'c' --> tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                        [-1.1433,  1.4024,  1.2703, -0.2610],
                        [-0.5595, -0.7062,  0.9987,  0.5466]])

In [4]:
t.a

tensor([[-0.4931,  0.2570, -0.5540],
        [-0.7806,  0.4787,  2.6710]])

In [5]:
%timeit t.a

62.9 ns ± 0.045 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7e2c8a1e80>
├── 'a' --> tensor([[ 1.1356,  0.4941, -2.0908],
│                   [-0.0435, -0.3492, -1.2834]])
└── 'x' --> <FastTreeValue 0x7f7e2c8a1f70>
    └── 'c' --> tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                        [-1.1433,  1.4024,  1.2703, -0.2610],
                        [-0.5595, -0.7062,  0.9987,  0.5466]])

In [7]:
%timeit t.a = new_value

67.5 ns ± 0.0258 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4931,  0.2570, -0.5540],
               [-0.7806,  0.4787,  2.6710]]),
    x: Batch(
           c: tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466]]),
       ),
)

In [10]:
b.a

tensor([[-0.4931,  0.2570, -0.5540],
        [-0.7806,  0.4787,  2.6710]])

In [11]:
%timeit b.a

53 ns ± 0.053 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5196,  0.6303,  0.3917],
               [-0.6195,  0.2363,  0.3983]]),
    x: Batch(
           c: tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.707 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

861 ns ± 0.976 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 54.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7d8716ed00>
├── 'a' --> tensor([[[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]],
│           
│                   [[-0.4931,  0.2570, -0.5540],
│                    [-0.7806,  0.4787,  2.6710]]])
└── 'x' --> <FastTreeValue 0x7f7d87158850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7d80534f70>
├── 'a' --> tensor([[-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710],
│                   [-0.4931,  0.2570, -0.5540],
│                   [-0.7806,  0.4787,  2.6710]])
└── 'x' --> <FastTreeValue 0x7f7d8716ee50>
    └── 'c' --> tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                        [-1.1433,  1.4024,  1.2703, -0.2610],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 96.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7748, -0.5670, -0.3421, -1.0476],
                       [-1.1433,  1.4024,  1.2703, -0.2610],
                       [-0.5595, -0.7062,  0.9987,  0.5466]],
              
                      [[-0.7748, -0.5670, -0.3421, -1.0476],
                       [-1.1433,  1.4024,  1.2703, -0.2610],
                       [-0.5595, -0.7062,  0.9987,  0.5466]],
              
                      [[-0.7748, -0.5670, -0.3421, -1.0476],
                       [-1.1433,  1.4024,  1.2703, -0.2610],
                       [-0.5595, -0.7062,  0.9987,  0.5466]],
              
                      [[-0.7748, -0.5670, -0.3421, -1.0476],
                       [-1.1433,  1.4024,  1.2703, -0.2610],
                       [-0.5595, -0.7062,  0.9987,  0.5466]],
              
                      [[-0.7748, -0.5670, -0.3421, -1.0476],
                       [-1.1433,  1.4024,  1.2703, -0.2610],
                       [-0.5595, -0.7062,  0.9987,  0.5466]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466],
                      [-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466],
                      [-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466],
                      [-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466],
                      [-0.7748, -0.5670, -0.3421, -1.0476],
                      [-1.1433,  1.4024,  1.2703, -0.2610],
                      [-0.5595, -0.7062,  0.9987,  0.5466],
                      [-0.7748, -0.5670, -0.3421, -1.0476],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
